## Sanity check

In [1]:
from typing import Dict, Callable
import numpy as np

# ---------- constants used by the metrics ----------
EPS = 1e-9
DADAPY_GRID_RANGE_MAX = 64
RAND_SEED = 42

# ---------- optional dependencies ----------
HAS_DADAPY = False
try:
    from dadapy import Data
    HAS_DADAPY = True
except Exception:
    pass

HAS_SKDIM = False
try:
    # Only the estimators you actually use
    from skdim.id import MOM, TLE, CorrInt, FisherS, lPCA, MLE, MADA, KNN
    HAS_SKDIM = True
except Exception:
    pass

# IsoScore: fall back to covariance‑eigenvalue based isotropy if not installed
try:
    from isoscore import IsoScore
    _HAS_ISOSCORE = True
except Exception:
    _HAS_ISOSCORE = False

    class _IsoScoreFallback:
        @staticmethod
        def IsoScore(X: np.ndarray) -> float:
            C = np.cov(X.T, ddof=0)
            ev = np.linalg.eigvalsh(C)
            if ev.mean() <= 0 or ev[-1] <= 0:
                return 0.0
            # mean/peak eigenvalue ratio in [0,1]; higher ≈ more isotropic
            return float(np.clip(ev.mean() / ev[-1], 0.0, 1.0))

    IsoScore = _IsoScoreFallback()


# =============================== HELPERS ===============================
def _center(X: np.ndarray) -> np.ndarray:
    return X - X.mean(0, keepdims=True)


def _eigvals_from_X(X: np.ndarray) -> np.ndarray:
    """Eigenvalues of covariance up to a constant via SVD of centered X (descending)."""
    Xc = _center(X.astype(np.float32, copy=False))
    try:
        _, S, _ = np.linalg.svd(Xc, full_matrices=False)
        lam = (S ** 2).astype(np.float64)
        lam.sort()
        return lam[::-1]
    except Exception:
        return np.array([], dtype=np.float64)


def _jitter_unique(X: np.ndarray, eps: float = 1e-6) -> np.ndarray:
    """Add tiny noise if there are duplicate rows (helps NN-based estimators)."""
    try:
        if np.unique(X, axis=0).shape[0] < X.shape[0]:
            X = X + np.random.normal(scale=eps, size=X.shape).astype(X.dtype)
    except Exception:
        pass
    return X


# ========= Per-subsample single-value compute functions =========
# --- Isotropy (fast) ---
def _iso_once(X: np.ndarray) -> float:
    return float(IsoScore.IsoScore(X))


def _spect_once(X: np.ndarray) -> float:
    ev = np.linalg.eigvalsh(np.cov(X.T, ddof=0))
    return float(ev[-1] / (ev.mean() + 1e-9))


def _rand_once(X: np.ndarray, K: int = 2000) -> float:
    n = X.shape[0]
    if n < 2:
        return float("nan")
    rng = np.random.default_rng()
    K_eff = min(K, (n * (n - 1)) // 2)
    i = rng.integers(0, n, size=K_eff)
    j = rng.integers(0, n, size=K_eff)
    same = i == j
    if same.any():
        j[same] = rng.integers(0, n, size=same.sum())
    A, B = X[i], X[j]
    num = np.sum(A * B, axis=1)
    den = (np.linalg.norm(A, axis=1) * np.linalg.norm(B, axis=1) + 1e-9)
    return float(np.mean(np.abs(num / den)))


def _sf_once(X: np.ndarray) -> float:
    lam = _eigvals_from_X(X)
    if lam.size == 0:
        return float("nan")
    gm = np.exp(np.mean(np.log(lam + EPS)))
    am = float(lam.mean() + EPS)
    return float(gm / am)


def _vmf_kappa_once(X: np.ndarray) -> float:
    if X.shape[0] < 2:
        return float("nan")
    Xn = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-9)
    R = np.linalg.norm(Xn.mean(axis=0))
    d = Xn.shape[1]
    if R < 1e-9:
        return 0.0
    # standard closed-form approximation
    return float(max(R * (d - R**2) / (1.0 - R**2 + 1e-9), 0.0))


# --- Linear ID (fast) ---
def _pcaXX_once(X: np.ndarray, var_ratio: float) -> float:
    lam = _eigvals_from_X(X)
    if lam.size == 0:
        return float("nan")
    c = np.cumsum(lam)
    thr = c[-1] * var_ratio
    return float(np.searchsorted(c, thr) + 1)


def _pca95_once(X: np.ndarray) -> float:
    return _pcaXX_once(X, 0.95)


def _pca99_once(X: np.ndarray) -> float:
    return _pcaXX_once(X, 0.99)


def _erank_once(X: np.ndarray) -> float:
    lam = _eigvals_from_X(X)
    if lam.size == 0:
        return float("nan")
    p = lam / (lam.sum() + EPS)
    H = -(p * np.log(p + EPS)).sum()
    return float(np.exp(H))


def _pr_once(X: np.ndarray) -> float:
    lam = _eigvals_from_X(X)
    if lam.size == 0:
        return float("nan")
    s1 = lam.sum()
    s2 = (lam ** 2).sum()
    return float((s1 ** 2) / (s2 + EPS))


def _stable_rank_once(X: np.ndarray) -> float:
    lam = _eigvals_from_X(X)
    if lam.size == 0:
        return float("nan")
    return float(lam.sum() / (lam.max() + EPS))


# --- Non-linear / intrinsic-dim (heavy) ---
def _dadapy_twonn_once(X: np.ndarray) -> float:
    if not HAS_DADAPY:
        return float("nan")
    d = Data(coordinates=_jitter_unique(X))
    id_est, _, _ = d.compute_id_2NN()
    return float(id_est)


def _dadapy_gride_once(X: np.ndarray) -> float:
    if not HAS_DADAPY:
        return float("nan")
    d = Data(coordinates=_jitter_unique(X))
    d.compute_distances(maxk=DADAPY_GRID_RANGE_MAX)
    ids, _, _ = d.return_id_scaling_gride(range_max=DADAPY_GRID_RANGE_MAX)
    return float(ids[-1])


def _skdim_factory(name: str):
    """Return a factory that builds a fresh skdim estimator each call, or None."""
    if not HAS_SKDIM:
        return None
    mapping = {
        "mom": MOM,
        "tle": TLE,
        "corrint": CorrInt,
        "fishers": FisherS,
        "lpca": lPCA,
        "lpca99": lPCA,
        "mle": MLE,
        "mada": MADA,
        "knn": KNN,
    }
    cls = mapping.get(name)
    if cls is None:
        return None

    def _builder():
        if name == "lpca":      # FO variant
            return cls(ver="FO")
        elif name == "lpca99":  # ratio (0.99) variant
            return cls(ver="ratio", alphaRatio=0.99)
        else:
            return cls()

    return _builder


def _skdim_once_builder(name: str) -> Callable[[np.ndarray], float] | None:
    build = _skdim_factory(name)
    if build is None:
        return None

    def _once(X: np.ndarray) -> float:
        est = build()
        est.fit(_jitter_unique(X))
        return float(getattr(est, "dimension_", float("nan")))

    return _once


def _bs_layer_loop(
    rep_sub: np.ndarray,
    M: int,
    n_reps: int,
    compute_once: Callable[[np.ndarray], float],
):
    """
    Bootstrap: sample M points with replacement and apply
    compute_once(X_layer) -> scalar for each layer.
    rep_sub: (L, N, D)
    """
    L, N, D = rep_sub.shape
    rng = np.random.default_rng(RAND_SEED)
    A = np.full((n_reps, L), np.nan, np.float32)
    for r in range(n_reps):
        idx = rng.integers(0, N, size=M)
        for l in range(L):
            X = rep_sub[l, idx].astype(np.float32, copy=False)
            try:
                A[r, l] = float(compute_once(X))
            except Exception:
                A[r, l] = np.nan
    mu = np.nanmean(A, axis=0).astype(np.float32)
    lo = np.nanpercentile(A, 2.5, axis=0).astype(np.float32)
    hi = np.nanpercentile(A, 97.5, axis=0).astype(np.float32)
    return mu, lo, hi


FAST_ONCE: Dict[str, Callable[[np.ndarray], float]] = {
    "iso": _iso_once,
    "spect": _spect_once,
    "rand": _rand_once,
    "sf": _sf_once,
    "vmf_kappa": _vmf_kappa_once,
    "erank": _erank_once,
    "pr": _pr_once,
    "stable_rank": _stable_rank_once,
}

# heavy metric registry
HEAVY_ONCE: Dict[str, Callable[[np.ndarray], float] | None] = {
    "twonn": _dadapy_twonn_once,
    "gride": _dadapy_gride_once,
    "mom": _skdim_once_builder("mom"),
    "tle": _skdim_once_builder("tle"),
    "corrint": _skdim_once_builder("corrint"),
    "fishers": _skdim_once_builder("fishers"),
    "lpca": _skdim_once_builder("lpca"),
    "lpca95": _skdim_once_builder("lpca95"),
    "lpca99": _skdim_once_builder("lpca99"),
    "mle": _skdim_once_builder("mle"),
    "mada": _skdim_once_builder("mada"),
    "knn": _skdim_once_builder("knn"),
}

LABELS = {
    # Isotropy
    "iso": "IsoScore",
    "spect": "Spectral Ratio",
    "rand": "RandCos |μ|",
    "sf": "Spectral Flatness",
    "vmf_kappa": "vMF κ",
    # Linear ID
    "erank": "Effective Rank",
    "pr": "Participation Ratio",
    "stable_rank": "Stable Rank",
    "lpca95": "lPCA95",
    "lpca99": "lPCA99",
    "lpca": "lPCA FO",
    # Non-linear
    "twonn": "TwoNN ID",
    "gride": "GRIDE",
    "mom": "MOM",
    "tle": "TLE",
    "corrint": "CorrInt",
    "fishers": "FisherS",
    "mle": "MLE",
    "mada": "MADA",
    "knn": "KNN",
}

PLOT_ORDER = (
    "iso", "sf", "vmf_kappa", "spect", "rand",
    "erank", "pr", "stable_rank", "lpca95", "lpca99", "lpca",
    "twonn", "gride", "mom", "tle", "corrint", "fishers",
    "mle", "mada", "knn",
)

ALL_METRICS = list(PLOT_ORDER)
# e.g. for quick tests:
# ALL_METRICS = ["gride"]


In [7]:
# ============================================
# 3. SYNTHETIC MANIFOLDS
# ============================================

def make_isotropic_gaussian(n=4000, d=16):
    rng = np.random.default_rng(RAND_SEED)
    return rng.normal(size=(n, d)).astype(np.float32)

def make_linear_subspace_gaussian(n=4000, ambient=64, intrinsic=4, noise=1e-3):
    rng = np.random.default_rng(RAND_SEED)
    Q, _ = np.linalg.qr(rng.normal(size=(ambient, intrinsic)))
    Z = rng.normal(size=(n, intrinsic))
    X = Z @ Q.T
    X += rng.normal(scale=noise, size=X.shape)
    return X.astype(np.float32)

def make_anisotropic_gaussian(n=4000, d=16, cond=100.0):
    rng = np.random.default_rng(RAND_SEED)
    ev = np.geomspace(1, 1/cond, d)
    Q, _ = np.linalg.qr(rng.normal(size=(d,d)))
    A = Q @ np.diag(np.sqrt(ev))
    Z = rng.normal(size=(n,d))
    return (Z@A.T).astype(np.float32)

def make_swiss_roll(n=4000, noise=0.05):
    rng = np.random.default_rng(RAND_SEED)
    t = rng.uniform(1.5*np.pi, 4.5*np.pi, size=n)
    h = rng.uniform(-1, 1, size=n)
    X = np.zeros((n,3), np.float32)
    X[:,0] = t*np.cos(t)
    X[:,1] = h
    X[:,2] = t*np.sin(t)
    X += noise*rng.normal(size=X.shape).astype(np.float32)
    return X

def default_manifolds():
    return {
        "iso_gauss_d16": {
            "X": make_isotropic_gaussian(4000,16),
            "true_id": 16,
            "true_iso": "isotropic"
        },
        "lowrank_d4_in_d64": {
            "X": make_linear_subspace_gaussian(),
            "true_id": 4,
            "true_iso": "isotropic in 4D subspace"
        },
        "anisotropic_cond100_d16": {
            "X": make_anisotropic_gaussian(),
            "true_id": 16,
            "true_iso": "anisotropic"
        },
        "swiss_roll_id2": {
            "X": make_swiss_roll(),
            "true_id": 2,
            "true_iso": "curved 2D manifold"
        }
    }



In [8]:
import numpy as np
import pandas as pd



def eval_metrics_on_manifolds(
    manifolds: dict,
    metrics: list[str] | None = None,
) -> pd.DataFrame:
    """
    Evaluate all (or a chosen subset of) metrics on each manifold.

    manifolds: output of default_manifolds()
               dict[name] -> {"X": array, "true_id": ..., "true_iso": ...}
    metrics:   list of metric names; if None, use all available in FAST_ONCE/HEAVY_ONCE
    """
    # Decide which metrics to run
    if metrics is None:
        metrics = []
        for name, fn in FAST_ONCE.items():
            if callable(fn):
                metrics.append(name)
        for name, fn in HEAVY_ONCE.items():
            if callable(fn):
                metrics.append(name)

    rows = []
    for mname, info in manifolds.items():
        X = info["X"]
        true_id = info.get("true_id")
        true_iso = info.get("true_iso")

        for metric in metrics:
            fn = FAST_ONCE.get(metric)
            if fn is None:
                fn = HEAVY_ONCE.get(metric)

            if fn is None or not callable(fn):
                val = np.nan
            else:
                try:
                    val = fn(X)
                except Exception:
                    val = np.nan

            rows.append(
                {
                    "manifold": mname,
                    "metric": metric,
                    "value": float(val) if np.isfinite(val) else np.nan,
                    "true_id": true_id,
                    "true_iso": true_iso,
                }
            )

    return pd.DataFrame(rows)


manifolds = default_manifolds()
df_metrics = eval_metrics_on_manifolds(manifolds)

display(df_metrics)

pivot = df_metrics.pivot(index="manifold", columns="metric", values="value")
display(pivot)


,manifold,metric,value,true_id,true_iso
0,iso_gauss_d16,iso,0.912916,16,isotropic
1,iso_gauss_d16,spect,1.095391,16,isotropic
2,iso_gauss_d16,rand,0.202039,16,isotropic
3,iso_gauss_d16,sf,0.997926,16,isotropic
4,iso_gauss_d16,vmf_kappa,0.293029,16,isotropic
...,...,...,...,...,...
71,swiss_roll_id2,lpca,2.000000,2,curved 2D manifold
72,swiss_roll_id2,lpca99,2.000000,2,curved 2D manifold
73,swiss_roll_id2,mle,2.026295,2,curved 2D manifold
74,swiss_roll_id2,mada,2.086253,2,curved 2D manifold


metric,corrint,erank,fishers,gride,iso,knn,lpca,lpca99,mada,mle,mom,pr,rand,sf,spect,stable_rank,tle,twonn,vmf_kappa
manifold,,,,,,,,,,,,,,,,,,,
anisotropic_cond100_d16,8.894571,8.573940,7.989226,9.010083,0.238264,3.0,10.0,14.0,10.709117,10.379160,9.194894,6.543065,0.300324,0.429625,4.197017,3.812231,10.814244,11.591547,0.345746
iso_gauss_d16,11.390293,15.966874,16.049872,12.216172,0.912916,2.0,16.0,16.0,14.097245,13.415234,12.508605,15.933997,0.202039,0.997926,1.095391,14.606650,13.713807,14.378677,0.293029
lowrank_d4_in_d64,3.897166,3.996816,3.992993,4.025223,0.059078,7.0,4.0,4.0,4.380554,4.070236,4.129300,3.992021,0.422056,0.000037,16.926774,3.780992,4.362872,4.039513,1.164865
swiss_roll_id2,1.920571,2.033507,1.940184,1.805512,0.609969,3.0,2.0,2.0,2.086253,2.026295,1.788330,1.995647,0.632341,0.287744,1.639427,1.829908,2.237239,2.498980,0.667884


In [ ]:
my_metrics = ["iso", "spect", "erank", "pr", "twonn", "gride", "mle"]
df_subset = eval_metrics_on_manifolds(default_manifolds(), metrics=my_metrics)


### Check convergence 